In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [15]:
data = pd.read_csv('NZairfares.csv')
data['Travel Date'] = pd.to_datetime(data['Travel Date'], infer_datetime_format=True)
data.head()


,Travel Date,Dep. airport,Dep. time,Arr. airport,Arr. time,Duration,Direct,Transit,Baggage,Airline,Airfare(NZ$)
0,2019-09-19,AKL,1:35 PM,CHC,3:00 PM,1h 25m,(Direct),NaN,NaN,Jetstar,111
1,2019-09-19,AKL,3:55 PM,CHC,5:20 PM,1h 25m,(Direct),NaN,NaN,Jetstar,111
2,2019-09-19,AKL,11:40 AM,CHC,1:05 PM,1h 25m,(Direct),NaN,NaN,Jetstar,132
3,2019-09-19,AKL,8:00 PM,CHC,9:25 PM,1h 25m,(Direct),NaN,NaN,Jetstar,132
4,2019-09-19,AKL,9:00 AM,CHC,10:25 AM,1h 25m,(Direct),NaN,NaN,Air New Zealand,133


In [16]:
print(f"Before: {len(data)}")

df = data[data['Airline'] == 'Air New Zealand']

df = df.drop(['Airline'], axis=1)

print(f"After: {len(df)}")

df.head()

Before: 162833
After: 157431


,Travel Date,Dep. airport,Dep. time,Arr. airport,Arr. time,Duration,Direct,Transit,Baggage,Airfare(NZ$)
4,2019-09-19,AKL,9:00 AM,CHC,10:25 AM,1h 25m,(Direct),NaN,NaN,133
7,2019-09-19,AKL,11:00 AM,CHC,12:25 PM,1h 25m,(Direct),NaN,NaN,163
8,2019-09-19,AKL,7:00 PM,CHC,8:25 PM,1h 25m,(Direct),NaN,NaN,163
10,2019-09-19,AKL,10:00 AM,CHC,11:25 AM,1h 25m,(Direct),NaN,NaN,193
11,2019-09-19,AKL,4:00 PM,CHC,5:25 PM,1h 25m,(Direct),NaN,NaN,193


In [17]:
print(df.isna().sum())
df = df.drop(['Transit', 'Baggage'], axis=1)
df.head()


Travel Date          0
Dep. airport        24
Dep. time            0
Arr. airport        24
Arr. time            0
Duration             0
Direct               0
Transit          36206
Baggage         157431
Airfare(NZ$)         0
dtype: int64


,Travel Date,Dep. airport,Dep. time,Arr. airport,Arr. time,Duration,Direct,Airfare(NZ$)
4,2019-09-19,AKL,9:00 AM,CHC,10:25 AM,1h 25m,(Direct),133
7,2019-09-19,AKL,11:00 AM,CHC,12:25 PM,1h 25m,(Direct),163
8,2019-09-19,AKL,7:00 PM,CHC,8:25 PM,1h 25m,(Direct),163
10,2019-09-19,AKL,10:00 AM,CHC,11:25 AM,1h 25m,(Direct),193
11,2019-09-19,AKL,4:00 PM,CHC,5:25 PM,1h 25m,(Direct),193


In [18]:
print(df.isna().sum())
df = df.dropna()


Travel Date      0
Dep. airport    24
Dep. time        0
Arr. airport    24
Arr. time        0
Duration         0
Direct           0
Airfare(NZ$)     0
dtype: int64


In [19]:
df.isna().sum()

Travel Date     0
Dep. airport    0
Dep. time       0
Arr. airport    0
Arr. time       0
Duration        0
Direct          0
Airfare(NZ$)    0
dtype: int64

In [20]:
df['Arrival and Departure'] = df['Arr. airport'] + ' - ' + df['Dep. airport']

df['Arrival and Departure'].unique().__len__()

32

Gives the amount of unique Arrival and Departure combinations

In [21]:
df.head()

,Travel Date,Dep. airport,Dep. time,Arr. airport,Arr. time,Duration,Direct,Airfare(NZ$),Arrival and Departure
4,2019-09-19,AKL,9:00 AM,CHC,10:25 AM,1h 25m,(Direct),133,CHC - AKL
7,2019-09-19,AKL,11:00 AM,CHC,12:25 PM,1h 25m,(Direct),163,CHC - AKL
8,2019-09-19,AKL,7:00 PM,CHC,8:25 PM,1h 25m,(Direct),163,CHC - AKL
10,2019-09-19,AKL,10:00 AM,CHC,11:25 AM,1h 25m,(Direct),193,CHC - AKL
11,2019-09-19,AKL,4:00 PM,CHC,5:25 PM,1h 25m,(Direct),193,CHC - AKL


In [22]:
df['Direct'].replace(['(Direct)', '(1 stop)', '(2 stops)', '(3 stops)'], [0, 1, 2, 3], inplace = True)
df['Dep. time'] = pd.to_datetime(df['Dep. time'], format='%I:%M %p').dt.strftime('%H:%M')
df['Arr. time'] = pd.to_datetime(df['Arr. time'], format='%I:%M %p').dt.strftime('%H:%M')
df['Dep. time'] = df['Dep. time'].str.split(':').str[0]
df['Arr. time'] = df['Arr. time'].str.split(':').str[0]
df['Duration'] = df['Duration'].str.replace("h", '*60').str.replace('m','*1').str.replace(' ','+').apply(eval)
df['Dep. time'] = df['Dep. time'].astype(int)
df['Arr. time'] = df['Arr. time'].astype(int)
print(df.dtypes)
df.head()

Travel Date              datetime64[ns]
Dep. airport                     object
Dep. time                         int64
Arr. airport                     object
Arr. time                         int64
Duration                          int64
Direct                            int64
Airfare(NZ$)                      int64
Arrival and Departure            object
dtype: object


,Travel Date,Dep. airport,Dep. time,Arr. airport,Arr. time,Duration,Direct,Airfare(NZ$),Arrival and Departure
4,2019-09-19,AKL,9,CHC,10,85,0,133,CHC - AKL
7,2019-09-19,AKL,11,CHC,12,85,0,163,CHC - AKL
8,2019-09-19,AKL,19,CHC,20,85,0,163,CHC - AKL
10,2019-09-19,AKL,10,CHC,11,85,0,193,CHC - AKL
11,2019-09-19,AKL,16,CHC,17,85,0,193,CHC - AKL


Converting Exact time to the hour, duration to minutes, direct to 0 1 2 or 3 depending on how many layovers

In [23]:
#X_train, X_test, y_train, y_test = train_test_split(subset_data.drop(columns = ['SalePrice']), subset_data['SalePrice'], test_size=0.25)
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 
['Travel Date', 'Dep. airport', 'Dep. time', 'Arr. airport', 'Arr. time', 'Arrival and Departure', 'Airfare(NZ$)']), df['Airfare(NZ$)'], test_size=0.25, random_state=10302000)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

118055
118055
39352
39352


In [24]:
model = LinearRegression(fit_intercept = True)
model.fit(X_train, y_train) 

# The following gives the R-square score
print('R^2: ', model.score(X_train, y_train)) 

# This is the coefficient Beta_1, ..., Beta_7
print('B1, B2: ', model.coef_)

# This is the coefficient Beta_0
print('B0: ', model.intercept_)

R^2:  0.27370029784417715
B1, B2:  [-1.00881225e-01  1.96070407e+02]
B0:  260.37139390206437


In [25]:
test_output = pd.DataFrame(model.predict(X_test), index = X_test.index, columns = ['pred_Airfare(NZ$)'])
# When extending to multiple features remove .array.reshape(-1, 1)
test_output.head()

,pred_Airfare(NZ$)
138367,421.133372
73098,419.620153
154693,381.789694
57609,433.743525
96806,444.336053


In [26]:
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_Airfare(NZ$)'] - test_output['Airfare(NZ$)']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)

Mean absolute error is 
108.16120456108199
